In [64]:
with open("input.txt", "r") as f:
    text = f.read()

In [65]:
print(f"length is: {len(text)}")

print("\n\nFirst 100 chars:\n--------")
print(text[:100])

length is: 1115394


First 100 chars:
--------
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [66]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"\n\nVocab size: {vocab_size}")
print("".join(chars))



Vocab size: 65

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [67]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [68]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(f"data type: {data.dtype}")
print(f"data shape: {data.shape}")
print(f"data: {data[:100]}")

data type: torch.int64
data shape: torch.Size([1115394])
data: tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [69]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [70]:
context_size = 8
train_data[:context_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [71]:
torch.manual_seed(1337)
batch_size = 4 # this is how many independent sequences will we process in parallel
block_size = 8 # this is max context lenth for predictions

def get_batch(split):
    """
    Generate a small bath of inputs x and targets y.
    """

    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # index of first value in the context
    x = torch.stack([data[i:i + block_size] for i in ix]) # context
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix]) # target, context shifted by one
    return x, y

xb, yb = get_batch("train")
print(f"xb shape: {xb.shape}")
print(f"xb: {xb}") # 32 independent examples

xb shape: torch.Size([4, 8])
xb: tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [ ]:
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        
        # creating an embedding table like we did above in PyTorch this time
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

        # creating a token embedding table of size vocab_size x vocab_size
        # nn.Embedding is a (thin) wrapper around a tensor of size vocab_size x vocab_size


    def forward(self, idx, targets=None):

        # idx and targets are both (B, ) tensor of integers
        logits = self.token_embedding_table(idx) 
        # will arrange in a batch by time by channel tensor (4 x 8 x embedding size) (B, T, C)
        # this will be interpreted as scores or logits of what comes next in the sequence
        # predicting what comes next based on the individual identity of a single token
        # token are not seeing any context currently except for themselves
        # each integer in our embedding table will pluck our a row from this table corresponding to its index

        # how well are we predicting the next character based on the logits?
        # logits is B x T x C tensor, cross entropy needs 
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T) # 
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions, in the logits, loss is ignored
            logits, loss = self(idx) # this self
            # focus on the last time step
            logits = logits[:, -1, :] # (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T + 1)
        return idx
            

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(f"out shape: {logits.shape}")
print(loss)

out shape: torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [ ]:
# one by one tensor holding a zero
idx_local = torch.zeros((1, 1), dtype=torch.long) # (1, 1), we kick of generation with 0 (newline character)

print(decode(m.generate(idx = idx_local, max_new_tokens=100)[0].tolist()))

# garbage output because totally random (untrained) model


SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [74]:
# create pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3) # this is a very fast learning rage

In [76]:
batch_size = 32
for steps in range(10_000):
    xb, yb = get_batch("train")

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True) # zero out the gradients
    loss.backward() # compute the gradients
    optimizer.step() # update the parameters

print(loss.item())

2.451270580291748


In [83]:
idx_local = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx = idx_local, max_new_tokens=100)[0].tolist()))


catlo pr itheewhimousorendouere t che tofo ixf an.
Whe! V:

Whifouthand nd athito cameralourgallain 
